In [12]:
import sys, os
repo_start = f'../'
sys.path.append(repo_start)

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from modules.utils.imports import *
from modules.loaders.format_data import format_data
from modules.genetic_algorithm.custom_deap_functions import *
from modules.genetic_algorithm.individual_to_surface import *
from modules.genetic_algorithm.ga_score_function import *

np.set_printoptions(threshold=sys.maxsize)

In [2]:
# Load and format training data
xt, u, v, shape_u, shape_v = format_data('../data/spikes_data.npz', plot=False)
u_triangle_mesh, v_triangle_mesh = lltriangle(u, v)

In [3]:
# Define the 3D function
def my_function(u, v):
    return (-923.43 * u) / (1 + 977.32 * u) + 98.61 * v * u**2 / (1 + 10.15 * u**2)
def true_function(u, v):
    return v * u**2 / (1 + 0.01 * u**2) - u

# Calculate the function values for each point in the meshgrid
z_ga = my_function(u_triangle_mesh, v_triangle_mesh)
z_true = true_function(u_triangle_mesh, v_triangle_mesh)


In [4]:
# calculate number of parameters
poly_terms = calculate_poly_terms(2, 2)
hill_terms = calculate_hill_terms(2, 2)
number_of_params = len(poly_terms) + 3 * 2 * len(hill_terms) 

best_individuals = np.zeros((100, number_of_params+1))
fitness_evolution = np.zeros((100, 100000+1))

for i in range(100):
    best_individuals[i] = np.load(f'/work/users/s/m/smyersn/elston/projects/kinetics_binns/development/equation_fitting/debugged_development/loss_functions/exact_terms/results/runs/best_ind_run_{i}.npz')['arr_0']
    fitness_evolution[i] = np.load(f'/work/users/s/m/smyersn/elston/projects/kinetics_binns/development/equation_fitting/debugged_development/loss_functions/exact_terms/results/runs/best_ind_run_{i}.npz')['arr_1']
    

best_individuals_sorted = sorted(best_individuals, key=lambda x: x[0])

In [5]:
ind = individual_to_surface(best_individuals_sorted[0][1:], u_triangle_mesh, v_triangle_mesh, poly_terms, hill_terms)

In [13]:
exact_terms_score_function(best_individuals_sorted[0][1:], u_triangle_mesh, v_triangle_mesh, poly_terms, hill_terms, 0, z_true, 0, 0, 'F_true')

(2496.751962601736,)

In [14]:
best_individuals_sorted[0][1:]

array([  0.        ,   0.        ,   0.        ,   3.64602083,
         0.        ,  -9.76378475,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   2.13386971,  80.54768427, 445.43083935,
       444.54724993, 468.57711701, 837.17783158, 713.99154811,
       525.22801506,   2.        ,   1.        ,   4.        ,
         2.        ,   3.        ,   1.        ,   4.        ,
         1.        ])

In [17]:
print(poly_terms)
print(hill_terms)

[(0,), (1,), (0, 0), (0, 1), (1, 1)]
[(0,), (1,), (0, 1), (1, 0)]


In [25]:
test = np.array([-1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0.01, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0])
ind2 = individual_to_surface(test, u_triangle_mesh, v_triangle_mesh, poly_terms, hill_terms)
exact_terms_score_function(test, u_triangle_mesh, v_triangle_mesh, poly_terms, hill_terms, 0, z_true, 0, 0, 'F_true')

(0.0,)

In [26]:
fig = make_subplots(rows=1, cols=1,
                    specs=[[{'type':'scene'}]],
                    subplot_titles=('F(A, B)'),
                    horizontal_spacing = 0)

fig.layout.annotations[0].update(y=0.8)
fig.update_annotations(font_size=24, font_color='#000000')


fig.add_trace(
    go.Surface(z=ind2, x=u_triangle_mesh, y=v_triangle_mesh,
                                colorscale='mint',
                                showscale=False,
                                colorbar=dict(
                                    x=1.15,
                                    len=0.5)),
    row=1, col=1)

scene_dict = dict( 
    xaxis_title='[A] (uM)',
    yaxis_title='[B] (uM)',
    zaxis_title='F',
    camera=dict(eye=dict(x=1, y=-2.5, z=1)),
    xaxis = dict(
        tick0 = 0,
        dtick = 2,
        tickfont = dict(size=18)),
    yaxis = dict(
        tick0 = 0.2,
        dtick = 0.4,
        tickfont = dict(size=18)),
    zaxis = dict(
        tick0 = 0,
        dtick = 2,
        tickfont = dict(size=18),
        range=[-1.5, 11]),)


fig.update_layout(autosize=True,
    width=1800, 
    height=800,
    font=dict(color = '#000000',
            size=20),
    
    scene=scene_dict)

fig.update_coloraxes(showscale=False)
fig.show()